In [2]:
import numpy as np
import pandas as pd
import sys 
eps = np.finfo(float).eps
from numpy import log2 as log
import random
from pprint import pprint

In [3]:
attributes = {
    'A1': ['b','a'],
    'A2': 'continuous',
    'A3': 'continuous',
    'A4': ['u','y','l','t'],
    'A5': ['g','p','gg'],
    'A6': ['c','d','cc','i','j','k','m','r','q','w','x','e','aa','ff'],
    'A7': ['v','h','bb','j','n','z','dd','ff','o'],
    'A8': 'continuous',
    'A9': ['t','f'],
    'A10': ['t','f'],
    'A11': 'continuous',
    'A12': ['t','f'],
    'A13': ['g','p','s'],
    'A14': 'continuous',
    'A15': 'continuous',
    'A16': ['+','-'],
}
attributes_copy = attributes.copy()
attributes_copy.pop('A16')
len(attributes_copy)

15

In [4]:
train_df= pd.read_csv("train.txt",sep = '\t', header = None, names = attributes.keys())
train_df
if '?' in train_df.values:
    print('Element ? exists in Dataframe')
else:
    print('Element ? does not exists in Dataframe')
# type(df)
# train_df.values

Element ? exists in Dataframe


In [5]:
# def fill_missing_values(df):
#     for key in df.keys():
#         df[key] = df[key].replace('?',df[key].mode()[0])
#     return df
        
# fill_missing_values(train_df)
# if '?' in train_df.values:
#     print('Element ? exists in Dataframe')
# else:
#     print('Element ? does not exists in Dataframe')

In [6]:
def fill_missing_values(df):
    # Filling in Missing Values
    for i in range(1, 16):
        attribute = 'A'+str(i)
        if type(attributes.get(attribute))!=list:
            mean_value = pd.to_numeric(df[df[attribute] != '?'][attribute]).mean()
            df[attribute] = pd.to_numeric(df[attribute].replace('?', mean_value))
        else:
            value_counts = df[attribute].value_counts()
            mode_value = None
            for index, value in value_counts.items():
                if index != '?':
                    mode_value = index
                    break
            if mode_value == None:
                print('Unable to find values other than \'?\'')
            df[attribute] = df[attribute].replace('?', mode_value)
fill_missing_values(train_df)
if '?' in train_df.values:
    print('Element ? exists in Dataframe')
else:
    print('Element ? does not exists in Dataframe')

Element ? does not exists in Dataframe


C:\Users\garga\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [9]:
def train_validation_split(df):
    df = df.sample(frac=1)
    size = int(0.8 * len(df))
    train_data = df[:size]
    val_data = df[size:]
    return train_data, val_data
train_set, val_set = train_validation_split(train_df)
print(train_set.shape, val_set.shape)

(392, 16) (98, 16)


In [181]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_treshold = 14
    for feature in df.columns:
#         print(feature)
        if feature != "A16":
            unique_values = df[feature].unique()
#             example_value = unique_values[0]

            if (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
#                 print('c')
            else:
                feature_types.append("continuous")
#                 print('conti')
    
    return feature_types
feature_types = determine_type_of_feature(train_df)
feature_types
# feature_types[int(find_winner(train_df)[0][1:])]

['categorical',
 'continuous',
 'continuous',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'continuous',
 'categorical',
 'categorical',
 'continuous',
 'categorical',
 'categorical',
 'continuous',
 'continuous']

In [182]:
threshold_dict = {}
train_df['A2']

0      30.83
1      24.50
2      20.17
3      32.08
4      45.83
       ...  
485    21.08
486    22.67
487    25.25
488    17.92
489    35.00
Name: A2, Length: 490, dtype: float64

In [183]:
# def continuous_to_categorical_data(data):
#     copy_data= data.copy()
#     for key in copy_data.keys()[:-1]:
#         if(type(attributes.get(key))!=list):
# #             print(key)
#             column_data = copy_data[key].astype(np.float)
# #             print("CD", column_data)
# #             print(type(copy_data[key]))
#             target_values = copy_data["A16"]
#             mydict = list(zip(column_data, target_values))
#             sorted_dic=sorted(mydict)
# #             print(sorted_dic)
#             new_column_data=pd.DataFrame(sorted_dic,columns=[0,"A16"])
# #             print("NCD", new_column_data[0][1])
#             my_column_changes = new_column_data["A16"].shift() != new_column_data["A16"]
# #             print("MCC", my_column_changes)
#             my_column_changes.iloc[0] = False
#             split_index= list(my_column_changes).index(True)
# #             print("Index", split_index)
#             #print("Index for splitting the attribute is: ", split_index)
#             #print(new_column_data[0].tolist())
#             #print("-----------")
#             #print(new_column_data["Approved"].tolist())
#             #max_info_gain,split_value=max_IG_split(new_column_data,new_column_data[0])
#             #print(new_column_data[0][split_index-1])
#             #print(new_column_data["Approved"][split_index-1])
#             #print(new_column_data[0][split_index])
#             #print(new_column_data["Approved"][split_index])
#             split_value = (float)(new_column_data[0][split_index] +new_column_data[0][split_index-1])/2.0
#             print("Value", split_value)
#             threshold_dict[key]=split_value
#             copy_data.loc[(column_data >= split_value), key] = 'Val >='+str(split_value)
#             copy_data.loc[(column_data < split_value), key] = 'Val <'+str(split_value)
# #             copy_data[key]=np.where(new_column_data >= split_value,'Val >='+str(split_value), 'Val <'+str(split_value))
           
#     return copy_data
# transformed_data = continuous_to_categorical_data(train_df)
# transformed_data

In [184]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

def entropy_continuous(df,data_portion):
    data_portion_count = len(data_portion)
#     print("In entropy continuous:" , data_portion_count)
    output = {}
    entropy = 0
    for x,y in data_portion:
        if y in output:
            output[y].append((x,y))
        else:
            output[y] = [(x,y)]
#     print("In entropy continuous:", output)
#     print("In entropy continuous:", len(output.get('+')))
    if ('+' in output.keys()):
        pos_labels_count = len(output.get('+'))
    else:
        pos_labels_count = 0
    neg_labels_count = data_portion_count - pos_labels_count
    pos_fraction = pos_labels_count/(data_portion_count+eps)
    pos_entropy = -pos_fraction*log(pos_fraction+eps)
    neg_fraction = neg_labels_count/(data_portion_count+eps)
    neg_entropy = -neg_fraction*log(neg_fraction+eps)
    entropy = pos_entropy + neg_entropy
    return entropy

def find_entropy_attribute(df,key):
    if(type(attributes.get(key))==list):
        Class = df.keys()[-1]   #To make the code generic, changing target variable class name
        target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
        variables = df[key].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
        entropy2 = 0
        threshold = None
        threshold_dict[key] = threshold
        for variable in variables:
            entropy = 0
            for target_variable in target_variables:
                num = len(df[key][df[key]==variable][df[Class] == target_variable])
                den = len(df[key][df[key]==variable])
                fraction = num/(den+eps)
                entropy += -fraction*log(fraction+eps)
            fraction2 = den/len(df)
            entropy2 += fraction2*entropy
#             entropy2 += -fraction2*entropy
    else:
        threshold = 0
        min_entropy = sys.maxsize
        entropy2 = 0
        threshold = None
        zipped = list(zip(df[key], df[df.keys()[-1]]))
        result = sorted(zipped, key = lambda x: x[0])
#         print(result)
        for i in range(len(result)):
            entropy2 = 0
            if(i!=0 and result[i][0]!=result[i-1][0]):      #check for all possible cases
                mid_value = (float(result[i][0])+float(result[i-1][0]))/2.0
#                 print(mid_value)
                left_part = result[0:i]
                left_part_count = len(left_part)
                if(left_part_count == 0):
                    continue
#                 print(left_part)
                entropy_left = entropy_continuous(df,left_part)
                left_fraction = len(left_part)/len(df)
                weighted_entropy_left = left_fraction*entropy_left
                entropy2+= weighted_entropy_left
#                 print(left_part_count)
                right_part = result[i:]
                right_part_count = len(right_part)
                if(right_part_count != 0):
                    entropy_right = entropy_continuous(df,right_part)
                    right_fraction = len(right_part)/len(df)
                    weighted_entropy_right = right_fraction*entropy_right
                    entropy2+= weighted_entropy_right
                    if(entropy2 < min_entropy):
                        min_entropy = entropy2
                        threshold = mid_value
        entropy2 = min_entropy
        threshold_dict[key] = threshold
        #         print(right_part_count)
                
#             else:
#                 print("Do nothing:", i)
    return (entropy2,threshold)
#     return abs(entropy2)
    

def find_winner(df, attributes_copy):
    Entropy_att = []
    IG = []
    for key in attributes_copy.keys():
#         print(key)
#         Entropy_att.append(find_entropy_attribute(df,key))
        entropy_attribute, threshold = find_entropy_attribute(df, key)
#         print(entropy_attribute, threshold)
        IG.append(find_entropy(df)-entropy_attribute)
    return list(attributes_copy)[np.argmax(IG)], threshold_dict.get(list(attributes_copy)[np.argmax(IG)])

# entropy_target = find_entropy(df)-find_entropy_attribute(df,'A6')
# entropy_target
# train_df.keys()[:-1]

In [164]:
# def find_entropy(df):
#     Class = df.keys()[-1]   #To make the code generic, changing target variable class name
#     entropy = 0
#     values = df[Class].unique()
#     for value in values:
#         fraction = df[Class].value_counts()[value]/len(df[Class])
#         entropy += -fraction*np.log2(fraction)
#     return entropy

# def entropy_continuous(df,data_portion):
#     data_portion_count = len(data_portion)
# #     print("In entropy continuous:" , data_portion_count)
#     output = {}
#     entropy = 0
#     for x,y in data_portion:
#         if y in output:
#             output[y].append((x,y))
#         else:
#             output[y] = [(x,y)]
# #     print("In entropy continuous:", output)
# #     print("In entropy continuous:", len(output.get('+')))
#     if ('+' in output.keys()):
#         pos_labels_count = len(output.get('+'))
#     else:
#         pos_labels_count = 0
#     neg_labels_count = data_portion_count - pos_labels_count
#     pos_fraction = pos_labels_count/(data_portion_count+eps)
#     pos_entropy = -pos_fraction*log(pos_fraction+eps)
#     neg_fraction = neg_labels_count/(data_portion_count+eps)
#     neg_entropy = -neg_fraction*log(neg_fraction+eps)
#     entropy = pos_entropy + neg_entropy
#     return entropy

# def find_entropy_attribute(df,key):
#     Class = df.keys()[-1]   #To make the code generic, changing target variable class name
#     target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
#     variables = df[key].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
#     entropy2 = 0
#     threshold = None
#     for variable in variables:
#         entropy = 0
#         for target_variable in target_variables:
#             num = len(df[key][df[key]==variable][df[Class] == target_variable])
#             den = len(df[key][df[key]==variable])
#             fraction = num/(den+eps)
#             entropy += -fraction*log(fraction+eps)
#         fraction2 = den/len(df)
#         entropy2 += fraction2*entropy
#     return (entropy2,threshold)
    

# def find_winner(df, attributes_copy):
#     Entropy_att = []
#     IG = []
#     for key in attributes_copy.keys():
# #         print(key)
# #         Entropy_att.append(find_entropy_attribute(df,key))
#         entropy_attribute, threshold = find_entropy_attribute(df, key)
#         IG.append(find_entropy(df)-entropy_attribute)
#     return list(attributes_copy)[np.argmax(IG)], threshold

# # entropy_target = find_entropy(df)-find_entropy_attribute(df,'A6')
# # entropy_target
# # attributes_copy.keys()
# # list(attributes_copy)[0]

In [165]:
# (find_winner(train_df, attributes_copy)
# threshold_dict
entropy_attribute, threshold = find_winner(train_df, attributes_copy)
print(entropy_attribute, threshold)
len(attributes_copy)
# print("Value", threshold_dict.get('A9'))
# type(attributes.get('A9'))

A9 None


15

In [9]:
train_df['A16'].describe()

count     490
unique      2
top         -
freq      277
Name: A16, dtype: object

In [166]:
def check_label_purity(data):
    label_column = data.keys()[-1]
    unique_classes = np.unique(data[label_column])

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [167]:
def majority_class(data):
    
    label_column = data.keys()[-1]
    unique_classes, counts_unique_classes = np.unique(data[label_column], return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [168]:
majority_class(train_df)

'-'

In [2]:
# def convert_continuous_to_categorical(df,threshold_dict):
#     for key in threshold_dict:
#         print("Key value:", key)
#         df[key].astype(np.float)
#         df.loc[df[key] > threshold_dict.get(key), key] = 1
#         df.loc[df[key] <= threshold_dict.get(key), key] = 0
# #         df[key] = np.where(df[key] >= threshold_dict.get(key), 1, df[key])
# #         df[key] = np.where(df[key] < threshold_dict.get(key), 0, df[key])
#         print("Done")
# convert_continuous_to_categorical(df,threshold_dict)
# df['A2'].values
# type(threshold_dict.get('A2'))
# df['A14'] = df['A14'].astype(np.float)
# type(df['A2'])
# df['A14'].values
# threshold_dict.values()
# df

NameError: name 'df' is not defined

In [14]:
# entropy = find_entropy(train_df)
# entropy

0.987658900015369

In [37]:
# find_entropy_attribute(train_df,'A4')

(0.9638639717644055, None)

In [125]:
def split_data(df, split_column, split_value, type_of_feature):
    data = df.values    
    split_column_data = data[:,split_column]
        

#     type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "categorical": 
        data_above_split = data[split_column_data != split_value]
        data_below_split = data[split_column_data == split_value]
        
    # feature is continuous   
    else:
        print("yes")
        split_column_data =  split_column_data.astype(np.float)
        data_above_split = data[split_column_data >  split_value]
        data_below_split = data[split_column_data <= split_value]
    
    data_above_split = pd.DataFrame(data = data_above_split, columns = attributes.keys())
    data_below_split = pd.DataFrame(data = data_below_split, columns = attributes.keys())
    return data_above_split, data_below_split
data_above_split, data_below_split = split_data(train_df,1,36.625,'continuous')
print("below", data_below_split.shape)
print("above", data_above_split.shape)
type(data_above_split)
data_above_split

yes
below (354, 16)
above (136, 16)


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,a,45.83,10.5,u,g,q,v,5,t,t,7,t,g,0,0,+
1,b,36.67,4.415,y,p,k,v,0.25,t,t,10,t,g,320,0,+
2,a,47.75,8,u,g,c,v,7.875,t,t,6,t,g,0,1260,+
3,a,47,13,u,g,i,bb,5.165,t,t,9,t,g,0,0,+
4,b,56.58,18.5,u,g,d,bb,15,t,t,17,t,g,0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,b,51.83,2.04,y,p,ff,ff,1.5,f,f,0,f,g,120,1,-
132,b,47.17,5.835,u,g,w,v,5.5,f,f,0,f,g,465,150,-
133,a,50.25,0.835,u,g,aa,v,0.5,f,f,0,t,g,240,117,-
134,a,41.58,1.04,u,g,aa,v,0.665,f,f,0,f,g,240,237,-


In [128]:
attributes.get('A1')

['b', 'a']

In [140]:
def decision_tree_algorithm(df, feature_types, attributes_copy, max_depth, min_samples=2, counter=0):
    
    # data preparations
    data = df
    if counter == 0:
        global COLUMN_LABELS, FEATURE_TYPES
        COLUMN_LABELS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
#         data = df.values
#     else:
#         data = df           
    
    
    # base cases
    if (check_label_purity(df)) or (len(df) < min_samples) or (counter == max_depth) or len(attributes_copy)==0:
        classification = majority_class(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
#         potential_splits = get_potential_splits(data)
#         split_column, split_value = determine_best_split(data, potential_splits)
#         data_above, data_below = split_data(data, split_column, split_value)
        
#         potential_splits = get_potential_splits(data)
        node, threshold = find_winner(data, attributes_copy)
        attributes_copy.pop(node)
        type_of_feature = feature_types[int(node[1:])]
        data_above, data_below = split_data(data, int(node[1:])-1, threshold, type_of_feature)
        
        # check for empty data
        if len(data_below) == 0:
            classification = majority_class(data)
            return classification
        
        if len(data_above) == 0:
            classification = majority_class(data)
            return classification
        
        # determine question
        feature_name = node
#        use this for type of feature = feature_types[int(find_winner(train_df)[0][1:])]
#         type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "categorical":
            values = attributes.get(feature_name)
            for split_value in values:
                question = "{} = {}".format(feature_name, split_value)
                sub_tree = {question: []}
            
        # feature is continuous
        else:
            question = "{} <= {}".format(feature_name, threshold)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, feature_types, attributes_copy, max_depth,  min_samples, counter)
        no_answer = decision_tree_algorithm(data_above, feature_types, attributes_copy, max_depth, min_samples, counter)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [141]:
tree = decision_tree_algorithm(train_df, feature_types, attributes_copy, max_depth=20)
pprint(tree)

0.9867798849009974 None
0.9500014251209004 36.625
0.9509894205998639 4.1875
0.9638639717644055 None
0.9638639717644055 None
0.9167768268934827 None
0.9845959106798634 None
0.9799538529493227 None
0.9633334834134044 356.0
yes


ValueError: could not convert string to float: 'v'

In [169]:
class Node():
    def __init__(self):
        self.val = []
        self.children = {}
        self.target_val = -1

    def add_child(self, cat, node):
        self.children[cat] = node

    def __repr__(self) -> str:
        return str(self.val) + " " + str(self.target_val)


def printTree(node, key, parent):
    print(parent, key, node)

    for key, child in node.children.items():
        printTree(child, key, node)

In [150]:
# attr = 'A4'
# feature_types[int(attr[1:]) - 1]
# train_df[train_df['A1'] == 'b']

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.250,t,t,1,f,g,202,0,+
2,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,120,0,+
3,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,360,0,+
5,b,36.67,4.415,y,p,k,v,0.250,t,t,10,t,g,320,0,+
7,b,23.25,1.000,u,g,c,v,0.835,t,f,0,f,s,300,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,b,36.42,0.750,y,p,d,v,0.585,f,f,0,f,g,240,3,-
484,b,40.58,3.290,u,g,m,v,3.500,f,f,0,t,s,400,0,-
485,b,21.08,10.085,y,p,e,h,1.250,f,f,0,f,g,260,0,-
488,b,17.92,0.205,u,g,aa,v,0.040,f,f,0,f,g,280,750,-


In [196]:
def build_tree(df, attribute_features, depth, maxDepth, feature_types):
    node = Node()
    
    target = len(df.columns)-1
    if depth == maxDepth or (len(attribute_features) == 0) or (len(df) <= 1) or check_label_purity(df) :
        tmp = df.iloc[:,target].mode()
#         classification = majority_class(df)
        if (len(tmp) == 0):
            node.target_val = 0
        else:
            node.target_val = tmp.iloc[0]
#         node.target_val = classification
        return node

    attr, thresh = find_winner(df, attribute_features)
#     print(attr)
    type_of_feature = feature_types[int(attr[1:]) - 1]
#     # categorial attribute
#     if thresh == 'None':
#         thresh = attributes.get(attr)

    
#     at_orig = [x for x in attributes if x != attr]
    attribute_features.pop(attr)
    # at_orig.remove(attr)

    if type_of_feature == 'categorical':
        thresh = attributes.get(attr)
        node.val = [attr, thresh]
        for val in thresh:
            node.add_child(val, build_tree(df[df[attr] == val], attribute_features, depth+1, maxDepth, feature_types))
    else:
        node.val = [attr, thresh]
        node.add_child('leq', build_tree(df[df[attr] <= thresh], attribute_features, depth+1, maxDepth, feature_types))
        node.add_child('ge', build_tree(df[df[attr] > thresh], attribute_features, depth+1, maxDepth, feature_types))

    return node

In [197]:
attributes_copy = attributes.copy()
attributes_copy.pop('A16')
node = build_tree(train_df,attributes_copy,0,20,feature_types)
printTree(node,"root","none")


none root ['A9', ['t', 'f']] -1
['A9', ['t', 'f']] -1 t ['A15', 385.5] -1
['A15', 385.5] -1 leq ['A8', 1.395] -1
['A8', 1.395] -1 leq ['A6', ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']] -1
['A6', ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']] -1 c ['A14', 220.0] -1
['A14', 220.0] -1 leq [] -
['A14', 220.0] -1 ge [] +
['A6', ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']] -1 d [] +
['A6', ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']] -1 cc [] +
['A6', ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']] -1 i ['A2', 28.125] -1
['A2', 28.125] -1 leq ['A3', 6.9175] -1
['A3', 6.9175] -1 leq [] +
['A3', 6.9175] -1 ge [] -
['A2', 28.125] -1 ge [] -
['A6', ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']] -1 j [] +
['A6', ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']] -1 k ['A11', 6.5] -1
['A11', 6.5] -1 leq

In [199]:
def evaluate(df, root):
    labels = []
    # tar = len(df.columns) - 1
    node = root
    # acc = 0
    for _, row in df.iterrows():
        while node.target_val == -1:
            b_attr = node.val[0]
            b_val = node.val[1]
            if isinstance(b_val, list):
                t_val = row[b_attr]
                node = node.children[t_val]
            else :
                t_val = row[b_attr]
                if (t_val <= b_val):
                    node = node.children['leq']
                else :
                    node = node.children['ge']
        # print(row[tar], node.target_val)
        # if(row[tar] == node.target_val):
        # 	acc += 1
        labels.append(node.target_val)
        node = root
    # print(acc/len(df))
    return labels
predicted_labels = evaluate(train_df,node)
predicted_labels

['+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '-',
 '-',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '-',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '-',
 '+',
 '+',
 '-',
 '+',
 '+',
 '-',
 '-',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '-',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '-',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '-',
 '-',
 '-'

In [202]:
def accuracy(orig, pred):
    num = len(pred)
    if(num != len(pred)):
        print('Error!! Num of labels are not equal.')
        return
    match = 0
    for i in range(len(orig)):
        o_label = orig[i]
        p_label = pred[i]
        if(o_label == p_label):
            match += 1
    print('***************\nAccuracy: ' + str(float(match)/num) + '\n***************')
original_labels = train_df['A16'].values
accuracy_score = accuracy(original_labels, predicted_labels)
accuracy_score

***************
Accuracy: 0.9510204081632653
***************


In [207]:
test_df= pd.read_csv("test.txt",sep = '\t', header = None, names = attributes.keys())
test_df
fill_missing_values(test_df)
test_prediction_label = evaluate(test_df,node)
test_original_label = test_df['A16'].values
test_accuracy_score = accuracy(test_original_label, test_prediction_label)
test_accuracy_score

***************
Accuracy: 0.5373134328358209
***************


In [136]:
train_df.columns

Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11',
       'A12', 'A13', 'A14', 'A15', 'A16'],
      dtype='object')

In [91]:
def ID3(data,features,target_attribute_name="A16",parent_node_class = None):
    """
    ID3 Algorithm: This function takes five paramters:
    1. data = the data for which the ID3 algorithm should be run --> In the first run this equals the total dataset
 
    2. originaldata = This is the original dataset needed to calculate the mode target feature value of the original dataset
    in the case the dataset delivered by the first parameter is empty

    3. features = the feature space of the dataset . This is needed for the recursive call since during the tree growing process
    we have to remove features from our dataset --> Splitting at each node

    4. target_attribute_name = the name of the target attribute

    5. parent_node_class = This is the value or class of the mode target feature value of the parent node for a specific node. This is 
    also needed for the recursive call since if the splitting leads to a situation that there are no more features left in the feature
    space, we want to return the mode target feature value of the direct parent node.
    """   
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    

#     # base cases
#     if (check_label_purity(df)) or (len(df) < min_samples) or (counter == max_depth):
#         classification = majority_class(data)
#         return classification
#     print("PNC", parent_node_class)
#     print("Attribute Feature len", len(features))
    classification = majority_class(data)
#     If all target_values have the same value, return this value
    if check_label_purity(data):
        print("Yes")
#         classification = majority_class(data)
        return classification
#     if len(np.unique(data[target_attribute_name])) <= 1:
#         return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        print("Yes1")
#         classification = majority_class(data)
        return classification
#         return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) == 0:
        print("Yes2")
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
#         parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        parent_node_class = classification
#         print("PNC inside", parent_node_class)
        #Select the feature which best splits the dataset
#         item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
#         best_feature_index = np.argmax(item_values)
#         best_feature = features[best_feature_index]
        
#         item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature, split_value = find_winner(data, features)
#         best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
#         features = [i for i in features if i != best_feature]
        
        features.pop(best_feature)
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data[data[best_feature] == value]
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,features,target_attribute_name,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)

In [92]:
import pprint

attributes_copy = attributes.copy()
attributes_copy.pop('A16')
# tree = buildTree(transformed_data)
ID3_tree = ID3(transformed_data,attributes_copy)
pprint.pprint(ID3_tree)
# len(attributes_copy)

Yes
Yes
Yes
Yes2
Yes
Yes
Yes
Yes2
Yes2
Yes2
Yes2
Yes2
Yes
Yes2
Yes
Yes
Yes
Yes2
Yes
Yes
Yes2
Yes2
{'A9': {'f': {'A6': {'aa': '-',
                     'c': {'A13': {'g': {'A12': {'f': {'A4': {'u': {'A1': {'a': '-',
                                                                           'b': {'A10': {'f': {'A7': {'h': '-',
                                                                                                      'v': {'A2': {'Val >=15.83': {'A3': {'Val >=0.0': {'A5': {'g': {'A8': {'Val >=0.0': {'A11': {'Val >=0.0': {'A14': {'Val >=0.0': {'A15': {'Val >=0.0': '-'}}}}}}}}}}}}}}}},
                                                                                         't': '-'}}}},
                                                              'y': '-'}},
                                                 't': '-'}},
                                   'p': '-',
                                   's': '-'}},
                     'cc': '-',
                     'd': '-',
        

In [44]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [60]:
def get_predicted_labels(df):
    predict_labels = []
    for i in range(len(df.index)):
        inst = df.iloc[i]
        prediction = predict(inst, ID3_tree)
        predict_labels.append(prediction)
    original_labels = df['A16'].values
    predict_labels = np.array(predict_labels)
    return predict_labels, original_labels
predict_labels, original_labels = get_predicted_labels(transformed_data)
print("Predicted", predict_labels)

Predicted ['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 

In [78]:
def accuracy(orig, pred):
    num = len(pred)
    if(num != len(pred)):
        print('Error!! Num of labels are not equal.')
        return
    match = 0
    for i in range(len(orig)):
        o_label = orig[i]
        p_label = pred[i]
        if(o_label == p_label):
            match += 1
    print('***************\nAccuracy: ' + str(float(match)/num) + '\n***************')

In [90]:
test_df= pd.read_csv("test.txt",sep = '\t', header = None, names = attributes.keys())
test_df
# if '?' in test_df.values:
#     print('Element ? exists in Dataframe')
# else:
#     print('Element ? does not exists in Dataframe')
# test_df_copy = fill_missing_values(test_df)
# if '?' in test_df_copy.values:
#     print('Element ? exists in Dataframe')
# else:
#     print('Element ? does not exists in Dataframe')
# test_df
# if '?' in test_df.values:
#     print('Element ? exists in Dataframe')
# else:
#     print('Element ? does not exists in Dataframe')
# transformed_test_data = continuous_to_categorical_data(test_df)
# transformed_test_data
# orig,pred = get_predicted_labels(df_test)
# print(len(orig), len(pred))

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,164,31285,+
1,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,80,1349,+
2,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,180,314,+
3,b,42.5,4.915,y,p,w,v,3.165,t,f,0,t,g,52,1442,+
4,b,22.08,0.830,u,g,c,h,2.165,f,f,0,t,g,128,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,b,32.33,3.500,u,g,k,v,0.500,f,f,0,t,g,232,0,-
130,b,?,0.040,y,p,d,v,4.250,f,f,0,t,g,460,0,-
131,b,47.83,4.165,u,g,x,bb,0.085,f,f,0,t,g,520,0,-
132,b,20,1.250,y,p,k,v,0.125,f,f,0,f,g,140,4,-


In [79]:
accuracy_score = accuracy(original_labels, predict_labels)
accuracy_score

***************
Accuracy: 0.5653061224489796
***************
